In [11]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pyodbc
import datetime as dt
from os.path import join
import roamability as rb

In [12]:
# Из базы данных OCS выгружаем TADIG

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME AS COUNTRY,n.NETWORK_NAME AS NETWORK,t.TADIG_CODE AS TADIG,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_tadig.head(3)

,COUNTRY,NETWORK,TADIG,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4
1,Afghanistan,Areeba/MTN,AFGAR,5
2,Afghanistan,Etisalat,AFG55,6


In [13]:
# Описываем класс данных Спонсора

class SponsorClass:
    
    def __init__(self, name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid,
                 reload_tariff_from_db = False, downloads = None, tariff_file_name = None):
        self.name = name
        self.sponsor_id = sponsor_id
        self.ocs_roaming_plan_id = ocs_roaming_plan_id
        self.roamdb_tarifflistid = roamdb_tarifflistid
        self.df_ocs_cost = DataFrame()
        self.df_roamdb = DataFrame()
        
        self.downloads = downloads
        self.tariff_file_name = tariff_file_name
        
        if reload_tariff_from_db:
            self.upload_data_from_ocs()
            self.upload_data_from_roamdb()
            df_temp = pd.merge(self.df_ocs_cost, self.df_roamdb, how='left', on='TADIG')
            self.df_tariff = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        else:
            self.prepare_pp()
        
    def ocs_tariff_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из OCS'''
        sql_srt=\
        '''
        SELECT t.tadig_code AS TADIG
        ,MO_CALL_RATE MOC,MT_CALL_RATE MTC,MO_SMS_RATE SMS,DATA_RATE DATA
        FROM ROAMING_PLAN_RULES r
        LEFT JOIN RDB_TADIG_CODES t
        ON r.network_id=t.network_id
        LEFT JOIN ROAMING_PLANS p
        ON r.roaming_plan_id=p.roaming_plan_id
        WHERE r.roaming_plan_id IN ({0})
        AND r.SPONSOR_ID = {1}
        AND p.deleted IS NULL
        AND r.end_date IS NULL
        AND r.is_discounted = 1
        '''.format(self.ocs_roaming_plan_id, self.sponsor_id)
        return sql_srt
    
    def roamdb_gen(self):
        '''Функция для подготовки SQL запроса для выгрузки тарифов из RoamDB'''
        sql_srt=\
        '''
        SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
        ,t.dataincrement_KB AS Data_Increment_KB
        ,IIF(t.support4G = 1, 'Yes', 'No') AS LTE,tc.value AS TADIG FROM Tariffs t
        INNER JOIN TadigCodes tc
        ON t.OperatorID = tc.OperatorID
        WHERE t.tarifflistid = {0}
        '''.format(self.roamdb_tarifflistid)
        return sql_srt
    
    def upload_data_from_ocs(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
        sql_srt = self.ocs_tariff_gen()
        self.df_ocs_cost = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def upload_data_from_roamdb(self):
        cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
        sql_srt = self.roamdb_gen()
        self.df_roamdb = pd.read_sql_query(sql_srt, cnxn)
        cnxn.close()
        
    def prepare_pp(self):
        '''Функция для подготовки DF из файла.
        Объединение с TADIG, COUNTRY и NETWORK_ID из OCS.
        Фильтрация по максимальному тарифу передачи данных.'''
        df_temp = pd.read_csv(join(self.downloads, self.tariff_file_name))
        df_temp = df_temp[df_temp['DATA'] <= discount_data_price_threshold]
        df_temp = pd.merge(df_temp, df_tadig, how='left', on='TADIG')
        self.df_tariff = df_temp

In [14]:
#######################################################################################
# Define variables here
#######################################################################################

combined_dict = {'Partner_Combined':'S1', 'P4_Combined':'S2', 'MB_Combined':'S4',
                 'SMART_Combined':'S5','S6_Combined':'S6','S8_Combined':'S8','S9_Combined':'S9'}

discount_data_price_threshold = 0.7 # USD
downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'

reload_tariff_from_db = False # False if the tariffs should be taken from files

# Output file names
file_compare_by_country_coverage = 'CompareByCountryCoverage.csv'
file_compare_by_lte_coverage = 'CompareByLteCoverage.csv'
file_compare_tariffs = 'CompareTariffs.csv'
file_country_min_tariffs = 'CountryMinTariff.xlsx'
file_country_min_tariffs_lte = 'CountryMinTariffLte.xlsx'
file_operator_min_tariffs_lte = 'OperatorMinTariffLte.xlsx'

# Files with the tariffs should contain the following fields:
# TADIG,MOC,MTC,SMS,DATA,Voice_Increment_SEC,Data_Increment_KB,LTE

# name, sponsor_id, ocs_roaming_plan_id, roamdb_tarifflistid, reload_tariff_from_db, downloads, tariff_file_name
s1_sponsor = SponsorClass('S1', 1, 267, 35, tariff_file_name = 'S1_20200506_FULL.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
s2_sponsor = SponsorClass('S2', 2, 268, 34, tariff_file_name = 'S2_20200701_FULL_USD_CAMEL_ONLY.csv',
                          reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
# s4_sponsor = SponsorClass('S4', 4, 329, 36, tariff_file_name = 'S4_20200801_FULL.csv',
#                           reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
# s5_sponsor = SponsorClass('S5', 5, 343, 32, tariff_file_name = 'S5_20200616_FULL_RB_COST.csv',
#                           reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)
# s6_sponsor = SponsorClass('S6', 6, 368, 39, tariff_file_name = 'S6_20200702_FULL_CAP_LTE_USD.csv',
#                           reload_tariff_from_db = reload_tariff_from_db, downloads = downloads)

# sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor]
sponsors = [s1_sponsor, s2_sponsor]

#######################################################################################

#######################################################################################

In [15]:
df_coverage = pd.read_csv(join(downloads, 'Package deal 2.0 Simtex Jul 21 2020 Coverage Ver2.csv'))
df_coverage.rename({'Country':'COUNTRY'}, axis=1, inplace=True)
df_coverage.A.replace({'v':'A'}, inplace=True)
df_coverage.B.replace({'v':'B'}, inplace=True)
df_coverage.C.replace({'v':'C'}, inplace=True)
df_coverage.drop('Region', axis=1, inplace=True)
df_coverage.head(3)

,COUNTRY,A,B,C
0,Oman,NaN,NaN,NaN
1,Mongolia,NaN,NaN,NaN
2,Benin,NaN,NaN,NaN


In [16]:
df_country_min = DataFrame({'COUNTRY':[]})

sponsor_name_list = []

for sponsor in sponsors:
    df_temp = sponsor.df_tariff.groupby('COUNTRY', as_index=False)['DATA'].min().rename({'DATA':f'{sponsor.name}'}, axis=1)
    df_country_min = pd.merge(df_country_min, df_temp, how='outer', on='COUNTRY')
    sponsor_name_list.append(sponsor.name)
    

df_country_min.set_index('COUNTRY', inplace=True)
df_country_min['SPONSOR'] = df_country_min.idxmin(axis=1)
df_country_min['MIN_PRICE'] = df_country_min.apply(lambda x: x[x.SPONSOR], axis=1)
df_country_min.drop(sponsor_name_list, axis=1, inplace=True)
df_country_min.reset_index(inplace=True)
df_country_min.head()

,COUNTRY,SPONSOR,MIN_PRICE
0,Albania,S1,0.0024
1,Andorra,S1,0.0300
2,Anguilla,S1,0.0257
3,Antigua and Barbuda,S1,0.0240
4,Argentina,S1,0.0140


In [17]:
df_country_min_coverage = pd.merge(df_country_min, df_coverage, how='left', on='COUNTRY')
df_country_min_coverage = df_country_min_coverage.loc[df_country_min_coverage[['A','B','C']].notna().any(axis=1)]
df_country_min_coverage.fillna('', inplace=True)
df_country_min_coverage.head()

,COUNTRY,SPONSOR,MIN_PRICE,A,B,C
0,Albania,S1,0.002400,,B,C
4,Argentina,S1,0.014000,,,C
5,Armenia,S1,0.004000,,B,C
6,Australia,S1,0.002200,A,B,C
7,Austria,S2,0.001437,A,B,C


In [18]:
def highlight_expensive(s):
    exp = ['']
    if s.MIN_PRICE > (25 / (10 * 1024)) and s.A:
        exp.append('background-color: #EF8282')
    elif s.A:
        exp.append('background-color: #85FAC4')
    else:
        exp.append('')
    if s.MIN_PRICE > (50 / (5 * 1024)) and s.B:
        exp.append('background-color: #EF8282')
    elif s.B:
        exp.append('background-color: #85FAC4')
    else:
        exp.append('')
    if s.MIN_PRICE > (75 / (5 * 1024)) and s.C:
        exp.append('background-color: #EF8282')
    elif s.C:
        exp.append('background-color: #85FAC4')
    else:
        exp.append('')
    return exp

df_country_min_coverage.style.apply(highlight_expensive, subset=['MIN_PRICE','A','B','C'], axis=1).hide_index()\
.to_excel(join(downloads, 'SimtexPackages.xlsx'), 'Sheet1', index=False)

df_country_min_coverage.head(15).style.apply(highlight_expensive, subset=['MIN_PRICE','A','B','C'], axis=1).hide_index()

COUNTRY,SPONSOR,MIN_PRICE,A,B,C
Albania,S1,0.0024,,B,C
Argentina,S1,0.014,,,C
Armenia,S1,0.004,,B,C
Australia,S1,0.0022,A,B,C
Austria,S2,0.0014375,A,B,C
Azerbaijan,S1,0.005,,B,C
Belarus,S1,0.003,,B,C
Belgium,S2,0.00161,A,B,C
Bosnia and Herzegov,S1,0.008,,,C
Brazil,S1,0.014,,,C


In [22]:
df_country_tadig = DataFrame()

for sponsor in sponsors:
    df_temp = df_country_min_coverage.loc[df_country_min_coverage.SPONSOR == sponsor.name]
    df_country_tadig = df_country_tadig.append(pd.merge(df_temp, sponsor.df_tariff[['COUNTRY','TADIG','DATA','NETWORK_ID']],
                                     how='left', on='COUNTRY'))

df_country_tadig[['A','B','C']] = df_country_tadig[['A','B','C']].replace(
    {'A':25 / (10 * 1024), 'B':50 / (5 * 1024), 'C':75 / (5 * 1024)}).replace({'':np.nan}).astype('float')

display(df_country_tadig.head())

,COUNTRY,SPONSOR,MIN_PRICE,A,B,C,TADIG,DATA,NETWORK_ID
0,Albania,S1,0.0024,NaN,0.009766,0.014648,ALBVF,0.0024,12
1,Albania,S1,0.0024,NaN,0.009766,0.014648,ALBAM,0.0090,9
2,Argentina,S1,0.0140,NaN,NaN,0.014648,ARGTM,0.0140,27
3,Argentina,S1,0.0140,NaN,NaN,0.014648,ARGCM,0.0250,25
4,Argentina,S1,0.0140,NaN,NaN,0.014648,ARGTP,0.0250,29


In [23]:
def select_min(s):
    price_list = []
    for val in [s.A, s.B, s.C]:
        if np.isnan(val):
            price_list.append(np.nan)
        elif s.MIN_PRICE == s.DATA:
            price_list.append(s.DATA)
        elif s.DATA > val:
            price_list.append(np.nan)
        elif s.DATA < val:
            price_list.append(s.DATA)
    return Series(price_list)

df_country_tadig[['A_COST','B_COST','C_COST']] = df_country_tadig.apply(select_min, axis=1)

###################################################################
# Remove TADIGs in countries which not allowed in all Plans
###################################################################

def check_abc_inconsistency(gr):
    """На вход подается сгруппированная таблица"""
    r = gr[['A_COST','B_COST','C_COST']].count() #значению r присваиваются значения группы
    if (r.A_COST != 0 and r.A_COST != r.C_COST):
        gr.dropna(subset=['A_COST','C_COST'], inplace=True)
    if (r.B_COST != 0 and r.B_COST != r.C_COST):
        gr.dropna(subset=['B_COST','C_COST'], inplace=True)
    return gr

df_country_tadig = df_country_tadig.groupby('COUNTRY', as_index=False).apply(check_abc_inconsistency)

###################################################################

df_country_tadig.dropna(subset=['A_COST','B_COST','C_COST'], how='all', inplace=True)

df_country_tadig.to_excel(join(downloads, 'SimtexPackagesTadig.xlsx'), 'Sheet1',
                          columns=['COUNTRY','TADIG','SPONSOR','A_COST','B_COST','C_COST','NETWORK_ID'],
                          index=False)

df_country_tadig.sort_values(by='COUNTRY').head(5)

COUNTRY SPONSOR  MIN_PRICE   A         B         C  TADIG    DATA  \
0 0    Albania      S1     0.0024 NaN  0.009766  0.014648  ALBVF  0.0024   
  1    Albania      S1     0.0024 NaN  0.009766  0.014648  ALBAM  0.0090   
1 2  Argentina      S1     0.0140 NaN       NaN  0.014648  ARGTM  0.0140   
2 5    Armenia      S1     0.0040 NaN  0.009766  0.014648  ARM01  0.0040   
  7    Armenia      S1     0.0040 NaN  0.009766  0.014648  ARM05  0.0090   

    NETWORK_ID  A_COST  B_COST  C_COST  
0 0         12     NaN  0.0024  0.0024  
  1          9     NaN  0.0090  0.0090  
1 2         27     NaN     NaN  0.0140  
2 5         30     NaN  0.0040  0.0040  
  7         33     NaN  0.0090  0.0090

In [43]:
# Compare the old and the new file country list

old_country_list = 'SimtexPackagesVer01.csv'

df_old_country_list = pd.read_csv(join(downloads, old_country_list), usecols=['COUNTRY','SPONSOR'])

df_old_new_country_compare = pd.merge(df_old_country_list, df_country_min_coverage[['COUNTRY','SPONSOR']],
         how='outer', on='COUNTRY', suffixes=['_OLD', '_NEW'])

df_old_new_country_compare.loc[df_old_new_country_compare.isna().any(axis=1)]\
.to_csv(join(downloads,'diff.csv'), index=False)

### Developments

In [15]:
df_temp = df_country_tadig.loc[:, ['COUNTRY','TADIG','SPONSOR','A_COST','B_COST','C_COST','NETWORK_ID']]
df_temp.head()

,COUNTRY,TADIG,SPONSOR,A_COST,B_COST,C_COST,NETWORK_ID
0,Albania,ALBVF,S1,NaN,0.0024,0.0024,12
1,Albania,ALBAM,S1,NaN,0.0090,0.0090,9
2,Argentina,ARGTM,S1,NaN,NaN,0.0140,27
3,Argentina,ARGCM,S1,NaN,NaN,NaN,25
4,Argentina,ARGTP,S1,NaN,NaN,NaN,29
